In [12]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

  Using cached https://github.com/chengs/tqdm/archive/colab.zip
  Created wheel for tqdm: filename=tqdm-4.28.1-py2.py3-none-any.whl size=47867 sha256=3d3042b9ffa7c0382bee8752afb858b46f3077d67d360e35283584b14c450f5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-cc7jd2_h/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.28.1 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
import re
import pandas as pd
from csv import writer
from keras.preprocessing.text import text_to_word_sequence

In [21]:
# tags from corpus
tagged_corpus = pd.read_csv("corpus_tags.tsv", sep="\t", names=["pmid", "start", "end", "word", "type", "umls_id"])
tagged_corpus = tagged_corpus.dropna()
tagged_corpus.iloc[610:640]

,pmid,start,end,word,type,umls_id
610,26867927,0,23,Radiofrequency ablation,T061,C0850292
611,26867927,27,59,posteroseptal accessory pathways,T023,C0447025
612,26867927,60,75,associated with,T080,C0332281
613,26867927,76,102,coronary sinus diverticula,"T019,T047",C3163894
614,26867927,103,135,Posteroseptal accessory pathways,T023,C0447025
615,26867927,143,158,associated with,T080,C0332281
616,26867927,161,193,coronary sinus (CS) diverticulum,"T019,T047",C3163894
617,26867927,227,251,clinical characteristics,T201,C0683325
618,26867927,253,260,mapping,T052,C1283195
619,26867927,265,291,ablation of these pathways,T061,C3275043


In [22]:
classes = []
with open("vocab_class.txt") as ftags:
    classes = [i.strip() for i in ftags.readlines()]
    ftags.close()
classes

['T047', 'T005', 'T007', 'T016']

In [0]:
 # Assisgns each word its respective tag with "O" as the default tag
sentence_count = 1
com_words = ['~']   #completes words when running through corpus

def get_words_tags(text, pmid,start=0):
    words = []
    tags = []  
    row_cont = []
    typ = "O"
    test = True
    global sentence_count
    df = tagged_corpus[(tagged_corpus["pmid"]==int(pmid)) & (tagged_corpus["end"] <= start + len(text)) & (tagged_corpus["start"] >= start)]
    indices = df.index.values
    text_split = text_to_word_sequence(text, filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
    for word in text_split:
      for i in indices:
        text2 = df.loc[i]["word"]
        text2_split = text_to_word_sequence(text2, filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
        if word in text2_split and set(df.loc[i]["type"].split(',')).intersection(classes):
          typ = df.loc[i]["type"].split(',')[0]
          #typ = "T047"
          if not word.isnumeric():
            #print(word,' # ',word in com_words)
            if word in com_words and test:
              test = True
            else:
              test = False
              com_words.append(word)
          break
        else:
          typ = "O"
      row_cont.append(["Sentence: " + str(sentence_count),word,typ])
    sentence_count = sentence_count + 1
    return row_cont,test



In [0]:
title_regex = re.compile("^\d+\|t\|")
abstract_regex = re.compile("^\d+\|a\|")
train_file = "train_T047.csv"
test_file = "test_T047.csv"

fr = open("corpus_pubtator.txt") #corpus file should be placed here
#dataset_file = "dataset_T047.csv" 
f1 = open(train_file,'a')
f1_writer = writer(f1)
first_row = ["Sentence #","Word","Tag"]
f1_writer.writerow(first_row)
f1.close()

f2 = open(test_file,'a')
f2_writer = writer(f2)
first_row = ["Sentence #","Word","Tag"]
f2_writer.writerow(first_row)
f2.close()

In [0]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        for row in list_of_elem:
          csv_writer.writerow(row)
        blank = ['','','']
        csv_writer.writerow(blank)

In [0]:
from tqdm import tnrange

abstract_count = 0
train_count = 0
test_count = 0
__ = len(fr.readlines())
fr.seek(0)

for i,_ in zip(fr.readlines(),tnrange(__, desc = 'Abstract',ncols='600px')):
    row_content = []
    line = i.strip()
    title_search = title_regex.search(line)
    abstract_search = abstract_regex.search(line)
    if title_search:
        title_line = title_regex.sub('',line)
    elif abstract_search:
        abstract = abstract_regex.sub('',line)
        pmid = int(abstract_search.group(0)[:-3])
        row_content,test = get_words_tags(title_line,pmid,0)
        if test:
          append_list_as_row(test_file, row_content)
          test_count += 1
        else:
           append_list_as_row(train_file, row_content)
           train_count += 1
        title_size = len(title_line) + 1                     # 1 for \n        
        for sentence in re.split("\. ", abstract):
            if len(sentence) == 0:
                continue
            _size = abstract.index(sentence) + title_size
            row_content,test = get_words_tags(sentence, pmid,_size)
            if test:
              append_list_as_row(test_file, row_content)
              test_count += 1
            else:
              append_list_as_row(train_file, row_content)
              train_count += 1
        abstract_count += 1
        if abstract_count%100 == 0:
            print("PMID {} written to file. Abstract Count: {} Train_count: {} Test_count: {}".format(pmid, abstract_count,train_count, test_count))
fr.close()
print("Completed")

PMID 27255600 written to file. Abstract Count: 100 Train_count: 100 Test_count: 1041
PMID 27288403 written to file. Abstract Count: 200 Train_count: 175 Test_count: 2028
PMID 27322653 written to file. Abstract Count: 300 Train_count: 236 Test_count: 3020
PMID 27355434 written to file. Abstract Count: 400 Train_count: 295 Test_count: 4071
PMID 27390849 written to file. Abstract Count: 500 Train_count: 360 Test_count: 5034
PMID 27430240 written to file. Abstract Count: 600 Train_count: 432 Test_count: 6093
PMID 27454637 written to file. Abstract Count: 700 Train_count: 522 Test_count: 7132
PMID 27482869 written to file. Abstract Count: 800 Train_count: 591 Test_count: 8178
PMID 27510784 written to file. Abstract Count: 900 Train_count: 654 Test_count: 9251
PMID 27543896 written to file. Abstract Count: 1000 Train_count: 702 Test_count: 10343
PMID 27575503 written to file. Abstract Count: 1100 Train_count: 749 Test_count: 11411
PMID 27609696 written to file. Abstract Count: 1200 Train_cou

In [0]:
print('__     ______    ___    __   _________')
print('| \   /  __  \  | \ \   | |  |   _____|')
print('|  |  | |  | |  | |\ \  | |  |  |_____')
print('|  |  | |__| |  | | \ \ | |  |  |_____')
print('|_/   \______/  |_|  \_\|_|  |________|')

In [0]:
print(com_words)